In [1]:
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username:  rh183
Password:  ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Batch run

In [12]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/',
    search_pattern_re='exp',
    max_depth=5,
    find_folders=True,
    find_files=False,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230804/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230726/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230708/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230808/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230711/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230725/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230721/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230722/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230717/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230806/scanimage_data/exp
/n/data1/h

In [13]:
dates_toUse = natsort.natsorted([Path(p).parts[-3] for p in paths_found])
dates_toUse

['20230702',
 '20230703',
 '20230705',
 '20230706',
 '20230707',
 '20230708',
 '20230709',
 '20230710',
 '20230711',
 '20230712',
 '20230713',
 '20230714',
 '20230715',
 '20230716',
 '20230717',
 '20230718',
 '20230719',
 '20230720',
 '20230721',
 '20230722',
 '20230723',
 '20230724',
 '20230725',
 '20230726',
 '20230727',
 '20230728',
 '20230729',
 '20230730',
 '20230731',
 '20230801',
 '20230802',
 '20230803',
 '20230804',
 '20230805',
 '20230806',
 '20230807',
 '20230808']

## Start a small interactive node

In [14]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=2G /bin/bash');

bash --pty -p interactive -t 0-00:30:00 --mem=2G /bin/ 

srun: job 16805675 queued and waiting for resources

srun: job 16805675 has been allocated resources

(base) [rh183@compute-e-16-233 ~]$ 


## Pull/update repo

In [15]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
send_expect('git pull');

c
_repos/NBAPhms/neurobio/sabatini/rich/github 
(base) [rh183@compute-e-16-233 NBAP]$ 
g
it pull

Already up-to-date.
(base) [rh183@compute-e-16-233 NBAP]$ 


## Activate environment

In [16]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

c
ini/rich/virtual_envs/FRms/neurobio/sabat 

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-233 NBA


## Prepare commands

In [17]:
for date in dates_toUse:
# for date in ['20230419']:

    mouse = 'mouse_0403L'
    # date = '20230501'

    name_slurm = mouse[-3:] + '_' + date[-4:]

    dir_s2p       = str(Path('/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output') / mouse / date / 'scanimage_data' / 'exp' / 'jobNum_0' / 'suite2p' / 'plane0' )

    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm') / mouse / date )


    name_job = 'jobNum_'


    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/dispatcher.py'))  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env


    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {dir_s2p} \
    {name_job} \
    {name_slurm} \
    {name_env}",
    }
    display(commands)
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    send_expect(commands['dispatch']);

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230702/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230702     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230702/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03L_0702     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

m.py     /n/data1/hms/neurobio/sabatini/ri/5a_make_dFoF_and_tqm/make_dFoF_and_tq 
0702     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03L_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230702/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0

Submitted batch job 16805736

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-233 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230703/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230703     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230703/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03L_0703     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

 
0703     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03L_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230703/make_dFoF_and_tqm.py
Submitting job: jobNum_ 0

Submitted batch job 16805739

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-233 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230705/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230705     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230705/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03L_0705     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

...

m.py     /n/data1/hms/neurobio/sabatinysis/5a_make_dFoF_and_tqm/make_dFoF_and_tq 
0806     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03L_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230806/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0
Submitted batch job 16805967

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-233 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230807/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230807     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230807/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03L_0807     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

 
0807     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03L_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230807/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0

Submitted batch job 16805975

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-233 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230808/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py     /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230808     /n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0403L/20230808/scanimage_data/exp/jobNum_0/suite2p/plane0     jobNum_     03L_0808     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

 
0808     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR    jobNum_     03L_ 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR


Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/5a_make_dFoF_and_tqm/make_dFoF_and_tqm.py to /n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm/mouse_0403L/20230808/make_dFoF_and_tqm.py

Submitting job: jobNum_ 0
Submitted batch job 16805976

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-e-16-233 NBA


### check on job

### Check if it worked

In [76]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701',
    search_pattern_re='tca_refit.h5',
    max_depth=5,
    find_folders=False,
    find_files=True,
))
paths_found

/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230601/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230512/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1

### Download results

In [83]:
dates = [Path(p).parts[-4] for p in paths_found]

In [85]:
paths_download = {d: p for d,p in zip(dates, paths_found)}

In [87]:
directory_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/tca_refit'

In [ ]:
[sftp.get(
    remotepath=p,
    localpath=str(Path(directory_save) / d / 'tca_refit.h5'),
    mkdirs=True,
) for d, p in paths_download.items()];

In [96]:
sftp.get_dir(
    source=r'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/',
    target=r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/',
)

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230531/parameters_batch.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230531/jobNum_0/params.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/jobNum_0/config.yaml   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230531/jobNum_0/config.yaml
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/jobNum_0/print_log_12089921.log   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230531/jobNum_0/print_log_12089921.log
downloading /n/data1/hms/neurobio/sabatini/rich/analysis

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/VQT_Analyzer.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/VQT_Analyzer.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/Dataset_videos.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/Dataset_videos.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/pipeline_basic.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230523/jobNum_0/pipeline_basic.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/run_info.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230523/jobNum_

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/jobNum_0/run_info.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230517/jobNum_0/run_info.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230517/jobNum_0/sbatch_config.sh
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/parameters_batch.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/jobNum_0/tca_refit.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/jobNum_0/tca_refit.h5
downloading /n

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/jobNum_0/print_log_12685323.log   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/jobNum_0/print_log_12685323.log
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/1_make_consistent_fr_factors.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/1_make_consistent_fr_factors.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/dispatcher.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/2023051

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230528/jobNum_0/params.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/config.yaml   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230528/jobNum_0/config.yaml
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/print_log_12021002.log   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230528/jobNum_0/print_log_12021002.log
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/cpu_tracker.csv   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230528/jobNum_0/cpu_tracker.csv
downloading /n/data1/hms/neurobio/sabatini/rich/an

...

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230516/jobNum_0/analysis_files/VQT_Analyzer.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230516/jobNum_0/analysis_files/VQT_Analyzer.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230516/jobNum_0/analysis_files/Dataset_videos.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230516/jobNum_0/analysis_files/Dataset_videos.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230516/jobNum_0/pipeline_basic.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230516/jobNum_0/pipeline_basic.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230516/jobNum_0/run_info.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230516/jobNum_

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/jobNum_0/run_info.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230509/jobNum_0/run_info.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230509/jobNum_0/sbatch_config.sh
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/parameters_batch.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/jobNum_0/tca_refit.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/jobNum_0/tca_refit.h5
downloading /n

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/aligned_TCA/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230529/aligned_TCA/dispatcher.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/aligned_TCA/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230529/aligned_TCA/notes.txt
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/pipeline_basic.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230529/pipeline_basic.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230529/dispatcher.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_2023

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230503/notes.txt
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230524/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230524/parameters_batch.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230524/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230524/jobNum_0/params.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230524/jobNum_0/config.yaml   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230524/jobNum_0/config.yaml
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/2023052

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/cpu_tracker.csv   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230502/jobNum_0/cpu_tracker.csv
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/print_log_12079182.log   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230502/jobNum_0/print_log_12079182.log
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/ROIs.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/ROIs.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/TCA.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/TCA.h5
download

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()